## Loading in the data from [Warframe Items](https://github.com/WFCD/warframe-items)

This uses the top level `data` folder from the above repo.

In [7]:
import json

with open('./data/json/Primary.json') as json_file:
    data_primary = json.load(json_file)

with open('./data/json/Secondary.json') as json_file:
    data_secondary = json.load(json_file)
    
with open('./data/json/Melee.json') as json_file:
    data_melee = json.load(json_file)
    
with open('./data/json/Sentinels.json') as json_file:
    data_sentinels = json.load(json_file)

## Generating itemComponents.js and item_names.py from the data

In [8]:
items = {}

def extract_useful_data(data):
    """Extract the components needed for each items and store them against the item name in the items dictionary.
    Also, return the names of items added to the dictionary from the current set of provided data.
    """
    item_names = []
    for item in data:
        if "components" in item.keys():
            item_components = [{"name": component["name"], "itemCount": component["itemCount"]} for component in item["components"]]
            item_components.append({"name": "Credits", "itemCount": item["buildPrice"]})
            items[item["name"]] = item_components
            item_names.append(item["name"])
    return item_names

# Extract the costs from all of the loaded in data sets
item_names_primary = extract_useful_data(data_primary)
item_names_secondary = extract_useful_data(data_secondary)
item_names_melee = extract_useful_data(data_melee)
item_names_sentinels = extract_useful_data(data_sentinels)
       
print(item_names_primary)
print(item_names_secondary)
print(item_names_melee)
print(item_names_sentinels)

# Save the items dictionary as a javascript file, so that it can be directly loaded by the page
with open("./js/itemComponents.js", "w") as f:
    f.write("var itemComponents = " + json.dumps(items))
    
# Generate a string of the data and save a python file. Horrendous bodge, but will be cut later when this notebook is
# later integrated with main.py.
item_names_string = ("item_names_primary = " +
                     json.dumps(item_names_primary) +
                     "\n\nitem_names_secondary = " +
                     json.dumps(item_names_secondary) +
                     "\n\nitem_names_melee = " +
                     json.dumps(item_names_melee) +
                     "\n\nitem_names_sentinels = " +
                     json.dumps(item_names_sentinels))
    
with open("./item_names.py", "w") as f:
    f.write(item_names_string)

['Acceltra', 'Amprex', 'Arca Plasmor', 'Argonak', 'Astilla', 'Attica', 'Basmu', 'Battacor', 'Baza', 'Baza Prime', 'Boar', 'Boar Prime', 'Boltor', 'Boltor Prime', 'Braton Prime', 'Braton Vandal', 'Bubonico', 'Burston', 'Burston Prime', 'Buzlok', 'Cernos', 'Cernos Prime', 'Convectrix', 'Corinth', 'Corinth Prime', 'Cortege', 'Corvas', 'Cryotra', 'Cyngas', 'Daikyu', 'Dera', 'Dera Vandal', 'Drakgoon', 'Dread', 'Dual Decurion', 'Exergis', 'Ferrox', 'Fluctus', 'Flux Rifle', 'Fulmin', 'Glaxion', 'Gorgon', 'Gorgon Wraith', 'Grakata', 'Grattler', 'Grinlok', 'Harpak', 'Hek', 'Helstrum', 'Hema', 'Hind', 'Ignis', 'Ignis Wraith', 'Imperator', 'Imperator Vandal', 'Javlok', 'Karak', 'Karak Wraith', 'Kohm', 'Komorex', 'Kuva Ayanga', 'Kuva Bramma', 'Kuva Chakkhurr', 'Kuva Drakgoon', 'Kuva Hind', 'Kuva Karak', 'Kuva Kohm', 'Kuva Ogris', 'Kuva Quartakk', 'Kuva Tonkor', 'Lanka', 'Larkspur', 'Latron', 'Latron Prime', 'Latron Wraith', 'Lenz', 'Miter', 'Morgha', 'Mutalist Cernos', 'Mutalist Quanta', 'Nagantak